In [1]:
import json
import os
import re
from pprint import pprint
import codecs

folder_in = 'json_cut'#'json_ridero_books' #'json_cut_chunks'
folder_out = 'parsed_json_cut'#'parsed_json_ridero_books' #'parsed_json_cut_chunks'

In [2]:
mystem_pos = [
    'A', 'ADV', 'ADVPRO', 'APRO', 'ANUM',
    'COM', 'CONJ', 'INTJ', 'NUM',
    'PART', 'PR', 'S', 'SPRO',
    'V', 'Vприч', 'Vинф', 'Vдеепр', 'UNK'
]
punct_pos= [
    'Pвнутр', 'Pгран', 'Pтире', 'Pкавычки', 'NL',
]

tire = ['–', '—', '-']
newline = '\n'
Pborders = ['.', '!', '?', "…"]
Pquotes = ['"', "'", '«', '»', '<', '>']
nl = 'NL'
digit = 'Digit'
pgran = 'Pгран'
pvnutr = 'Pвнутр'
lat = 'LAT'

In [3]:
ord('—'), ord('-')

(8212, 45)

In [4]:
all_pos = [*mystem_pos, *punct_pos, digit, nl, pgran, lat]

stop_3grams = set()
for p1 in all_pos:
    stop_3grams.add(" ".join([nl, p1, nl]))
    stop_3grams.add(" ".join([nl, 'Pтире', p1]))
    
    stop_3grams.add(" ".join([p1, lat, lat]))
    stop_3grams.add(" ".join([lat, p1, lat]))
    stop_3grams.add(" ".join([lat, lat, p1]))
    
    stop_3grams.add(" ".join([p1, digit, digit]))
    stop_3grams.add(" ".join([digit, p1, digit]))
    stop_3grams.add(" ".join([digit, digit, p1]))
    
    for p2 in all_pos:
        stop_3grams.add(" ".join([p1, nl, p2]))
        stop_3grams.add(" ".join([p1, pgran, p2]))
#         stop_3grams.add(" ".join([p1, pvnutr, p2]))
        
        stop_3grams.add(" ".join([lat, p1, p2]))
        stop_3grams.add(" ".join([p1, lat, p2]))
        stop_3grams.add(" ".join([p1, p2, lat]))
        
        stop_3grams.add(" ".join([digit, p1, p2]))
        stop_3grams.add(" ".join([p1, digit, p2]))
        stop_3grams.add(" ".join([p1, p2, digit]))

with open('stop3grams.txt', 'w') as f:
    f.write('\n'.join(sorted(stop_3grams)))

In [5]:
re.match('[а-яА-Я]', 'ри')


<re.Match object; span=(0, 1), match='р'>

In [6]:
%%time
if not os.path.exists(folder_out):
    os.makedirs(folder_out)

i = 0
for filename in sorted(os.listdir(folder_in)):
    i += 1
    print(i, filename)
    pos_tags = []
    
    with codecs.open(os.path.join(folder_in, filename), encoding='utf-8') as book:
        content = book.read()
        lines = [json.loads(l) for l in content.splitlines()]
        for line in lines:
            pos = 'IDK'
            end_nl = False
            if 'analysis' in line:
# могут быть пустыми {... 'analysis': [] ...}
                if not line['analysis']:
                    if re.match('[а-яА-Я]', line['text']):
                        pos = 'UNK'
                        pos_tags.append(pos)
                    elif any(c.isalpha() for c in line['text']):
                        pos = 'LAT'
                        pos_tags.append(pos)
                    else:
                        print('else', line)
                    continue
                    
                tags = re.split('[,=()"]',  line['analysis'][0]['gr'])
                pos = tags[0]
                if 'деепр' in tags:
                    pos = 'Vдеепр'
                elif 'прич' in tags:
                    pos = 'Vприч'
                elif 'инф' in tags:
                    pos = 'Vинф'
                pos_tags.append(pos)

            else:
#значит знаки препинания либо числа.
                text = line['text']
                text = text.replace('\xa0', ' ')
                text = text.replace('\\s', ' ')
                text = text.strip(' ')
                if text.endswith('\n'):
                    end_nl = True
                    text = text.rstrip('\n')
                if not text:
                    if end_nl:
                        pos_tags.append('NL')
                    continue

                elif re.match('[а-яА-Я]', text):
                    pos = 'UNK'
                    pos_tags.append(pos)
                elif any(c.isalpha() for c in line['text']):
                    pos = 'LAT'
                    pos_tags.append(pos)
                elif any(c.isdigit() for c in text):
                    pos = 'Digit'
                    pos_tags.append(pos)
                else:
                    text = [c for l in text.split() for c in l]
                    if not text:
                        print(line)
                    else:
                        for p in text:
                            if p in Pborders:
                                pos = 'Pгран'
                                pos_tags.append(pos)
                            elif p in Pquotes:
                                pos = 'Pкавычки'
                                pos_tags.append(pos)
                            elif p in tire:
                                pos = 'Pтире'
                                pos_tags.append(pos)
                            else:
                                pos = 'Pвнутр'
                                pos_tags.append(pos)
            if pos == 'IDK':
                print('idk', line)
            if end_nl:
                pos_tags.append('NL')
    with open(os.path.join(folder_out, filename), 'w') as wf:
        for pos in pos_tags:
            wf.write(pos)
            wf.write(' ')

1 100333.fb2.txt.json
2 101185.fb2.txt.json
3 102112.fb2.txt.json
4 102645.fb2.txt.json
5 102649.fb2.txt.json
6 103387.fb2.txt.json
7 103588.fb2.txt.json
8 10547.fb2.txt.json
9 105610.fb2.txt.json
else {'analysis': [], 'text': '№'}
else {'analysis': [], 'text': '№'}
else {'analysis': [], 'text': '№'}
else {'analysis': [], 'text': '№'}
else {'analysis': [], 'text': '№'}
else {'analysis': [], 'text': '№'}
else {'analysis': [], 'text': '№'}
10 105764.fb2.txt.json
11 1061.fb2.txt.json
12 106459.fb2.txt.json
else {'analysis': [], 'text': '¦'}
13 1068.fb2.txt.json
14 1070.fb2.txt.json
15 107104.fb2.txt.json
16 107401.fb2.txt.json
17 107403.fb2.txt.json
18 109209.fb2.txt.json
19 110929.fb2.txt.json
20 111375.fb2.txt.json
21 1122.fb2.txt.json
22 113370.fb2.txt.json
23 113372.fb2.txt.json
24 114298.fb2.txt.json
25 114299.fb2.txt.json
26 114591.fb2.txt.json
else {'analysis': [], 'text': '№'}
else {'analysis': [], 'text': '№'}
else {'analysis': [], 'text': '№'}
27 115721.fb2.txt.json
28 115826.fb

243 230081.fb2.txt.json
244 232264.fb2.txt.json
245 232269.fb2.txt.json
246 232857.fb2.txt.json
247 234281.fb2.txt.json
248 235106.fb2.txt.json
249 237783.fb2.txt.json
250 238059.fb2.txt.json
251 239744.fb2.txt.json
252 241406.fb2.txt.json
253 241407.fb2.txt.json
254 245158.fb2.txt.json
255 245162.fb2.txt.json
256 246296.fb2.txt.json
257 24680.fb2.txt.json
258 24693.fb2.txt.json
259 24695.fb2.txt.json
260 24698.fb2.txt.json
261 248853.fb2.txt.json
262 248855.fb2.txt.json
else {'analysis': [], 'text': '€'}
else {'analysis': [], 'text': '€'}
263 248856.fb2.txt.json
264 249472.fb2.txt.json
265 250244.fb2.txt.json
266 250335.fb2.txt.json
else {'analysis': [], 'text': '№'}
else {'analysis': [], 'text': '№'}
else {'analysis': [], 'text': '№'}
267 250556.fb2.txt.json
268 252791.fb2.txt.json
269 253788.fb2.txt.json
270 254496.fb2.txt.json
271 255292.fb2.txt.json
272 25875.fb2.txt.json
273 259579.fb2.txt.json
274 261423.fb2.txt.json
else {'analysis': [], 'text': '№'}
else {'analysis': [], 'text

520 53075.fb2.txt.json
521 53076.fb2.txt.json
522 53078.fb2.txt.json
523 53166.fb2.txt.json
524 532519.fb2.txt.json
525 533328.fb2.txt.json
526 53364.fb2.txt.json
527 53564.fb2.txt.json
else {'analysis': [], 'text': '°'}
else {'analysis': [], 'text': '°'}
else {'analysis': [], 'text': '°'}
528 53580.fb2.txt.json
529 535889.fb2.txt.json
530 53607.fb2.txt.json
531 536894.fb2.txt.json
532 539503.fb2.txt.json
533 56643.fb2.txt.json
534 57678.fb2.txt.json
535 57681.fb2.txt.json
536 58521.fb2.txt.json
537 61578.fb2.txt.json
538 61787.fb2.txt.json
539 61821.fb2.txt.json
540 61925.fb2.txt.json
541 65796.fb2.txt.json
542 65797.fb2.txt.json
543 65864.fb2.txt.json
544 65888.fb2.txt.json
545 66372.fb2.txt.json
546 66373.fb2.txt.json
547 66988.fb2.txt.json
548 67536.fb2.txt.json
549 67550.fb2.txt.json
550 67903.fb2.txt.json
551 68317.fb2.txt.json
552 68319.fb2.txt.json
553 68344.fb2.txt.json
554 68345.fb2.txt.json
555 69159.fb2.txt.json
556 69167.fb2.txt.json
557 69300.fb2.txt.json
558 69716.fb2.tx

In [7]:
text = ", —\n"
text = text.replace('\xa0', ' ')
text = text.replace('\\s', ' ')
text = text.strip(' ')
text

', —\n'

In [6]:
[c for l in '3р'.split() for c in l]

['3', 'р']

else {'analysis': [], 'text': '№'}
else {'analysis': [], 'text': '±'}
else {'analysis': [], 'text': '°'}
else {'analysis': [], 'text': '¦'}
else {'analysis': [], 'text': '√'}
else {'analysis': [], 'text': '©'}
else {'analysis': [], 'text': '®'}
else {'analysis': [], 'text': '€'}